# Izvajanje

Običajno bi po sintaksi jezika formalno podali še njegovo semantiko, torej pomen posameznih delov. Ker za to še nimamo ustreznih matematičnih orodij, bomo ravnali podobno kot pri večini programskih jezikov: napisali bomo implementacijo, torej program, ki prebere ukaze, zapisane v konkretni sintaksi, ter jih na nek (po vsej sreči smiselen) način izvede. Nato pa bomo proglasili, da je pomen programa v IMPu tisto, kar implementacija z njim naredi. Implementacijo bomo napisali v programskem jeziku [OCaml](https://ocaml.org), ki je eden najprikladnejših jezikov za implementacije programskih jezikov. Končnica `ML` namreč pomeni _meta-language_ oziroma metajezik, torej jezik za opis jezikov.

Spomnimo se abstraktne sintakse jezika IMP:

In [1]:
type location = Location of string

type exp =
  | Lookup of location
  | Int of int
  | Plus of exp * exp
  | Minus of exp * exp
  | Times of exp * exp

type bexp =
  | Bool of bool
  | Equal of exp * exp
  | Less of exp * exp
  | Greater of exp * exp

type cmd =
  | IfThenElse of bexp * cmd * cmd
  | WhileDo of bexp * cmd
  | Seq of cmd * cmd
  | Assign of location * exp
  | Skip

type location = Location of string


type exp =
    Lookup of location
  | Int of int
  | Plus of exp * exp
  | Minus of exp * exp
  | Times of exp * exp


type bexp =
    Bool of bool
  | Equal of exp * exp
  | Less of exp * exp
  | Greater of exp * exp


type cmd =
    IfThenElse of bexp * cmd * cmd
  | WhileDo of bexp * cmd
  | Seq of cmd * cmd
  | Assign of location * exp
  | Skip


in primerov

In [2]:
let exp1 = Times (Int 6, Int 7)
let exp2 = Minus (Lookup (Location "m"), Int 1)
let bexp = Greater (Lookup (Location "m"), Int 0)
let cmd1 = Assign (Location "m", exp2)
let cmd2 = WhileDo (bexp, cmd1)

val exp1 : exp = Times (Int 6, Int 7)


val exp2 : exp = Minus (Lookup (Location "m"), Int 1)


val bexp : bexp = Greater (Lookup (Location "m"), Int 0)


val cmd1 : cmd = Assign (Location "m", Minus (Lookup (Location "m"), Int 1))


val cmd2 : cmd =
  WhileDo (Greater (Lookup (Location "m"), Int 0),
   Assign (Location "m", Minus (Lookup (Location "m"), Int 1)))


## Aritmetični in logični izrazi

Pomen bomo najprej določili aritmetičnim izrazom, ki naj bi predstavljali cela števila. Ker izrazi lahko vsebujejo pomnilniške lokacije, mora funkcija za evalvacijo poleg izraza sprejeti tudi trenutno stanje pomnilnika, ki ga bomo predstavili kar s seznamom parov, ki danim lokacijam pripiše cela števila, na primer

In [3]:
let st1 = [(Location "m", 10); (Location "n", 0)]
let st2 = [(Location "m", 5)]

val st1 : (location * int) list = [(Location "m", 10); (Location "n", 0)]


val st2 : (location * int) list = [(Location "m", 5)]


In [4]:
let rec eval_exp st = function
  | Lookup l -> List.assoc l st
  | Int n -> n
  | Plus (e1, e2) -> eval_exp st e1 + eval_exp st e2
  | Minus (e1, e2) -> eval_exp st e1 - eval_exp st e2
  | Times (e1, e2) -> eval_exp st e1 * eval_exp st e2

val eval_exp : (location * int) list -> exp -> int = <fun>


Tako na primer velja

In [5]:
eval_exp [] exp1

- : int = 42


In [6]:
eval_exp st1 exp2

- : int = 9


In [7]:
eval_exp st2 exp2

- : int = 4


Podobno lahko definiramo funkcijo za evalvacijo logičnih izrazov, ki sprejme stanje in logični izraz ter vrne logično vrednost:

In [8]:
let eval_bexp st = function
  | Bool b -> b
  | Equal (e1, e2) -> eval_exp st e1 = eval_exp st e2
  | Less (e1, e2) -> eval_exp st e1 < eval_exp st e2
  | Greater (e1, e2) -> eval_exp st e1 > eval_exp st e2

val eval_bexp : (location * int) list -> bexp -> bool = <fun>


In [9]:
eval_bexp st1 bexp

- : bool = true


In [10]:
eval_bexp st2 bexp

- : bool = true


## Ukazi

Nazadnje definirajmo še funkcijo za evalvacijo ukazov. Funkcija sprejme stanje in ukaz, vrne pa končno vrednost stanja po izvršenem ukazu.

In [11]:
let rec eval_cmd st = function
  | IfThenElse (b, c1, c2) ->
      if eval_bexp st b then eval_cmd st c1 else eval_cmd st c2
  | WhileDo (b, c) ->
      (* eval_cmd st (IfThenElse (b, Seq (c, WhileDo (b, c)), Skip)) *)
      if eval_bexp st b then
        let st' = eval_cmd st c in
        eval_cmd st' (WhileDo (b, c))
      else st
  | Seq (c1, c2) ->
      let st' = eval_cmd st c1 in
      eval_cmd st' c2
  | Assign (l, e) -> (l, eval_exp st e) :: List.remove_assoc l st
  | Skip -> st

val eval_cmd : (location * int) list -> cmd -> (location * int) list = <fun>


In [12]:
eval_cmd st1 cmd1

- : (location * int) list = [(Location "m", 9); (Location "n", 0)]


In [13]:
eval_cmd st2 cmd1

- : (location * int) list = [(Location "m", 4)]


In [14]:
eval_cmd st1 cmd2

- : (location * int) list = [(Location "m", 0); (Location "n", 0)]


In [15]:
eval_cmd st2 cmd2

- : (location * int) list = [(Location "m", 0)]


## Vaje

### Naloga 1

Razmislite, kako bi dopolnili evalvator jezika IMP za:

1. logična veznika `&&` in `||`,

2. ukaz `SWITCH`, ki zamenja vrednosti dveh lokacij,

3. ukaz `FAIL`, ki prekine izvajanje programa.